In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#!pip install ydata_profiling
!pip install pandas-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 23.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=fa66c10167e5635d89e2ede2905cd43ebdc1ba93623e081b50e13ef038121352
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_ex

In [7]:
import duckdb
import glob
import time
import os
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
#import ydata_profiling
#from pandas_profiling import ProfileReport
#from ydata_profiling import ProfileReport
#import pandas_profiling
#from pandas_profiling import ProfileReport
#from pandas_profiling.utils.cache import cache_file

pd.set_option('display.max_rows', None) # for printing full dataframe

In [8]:
#Creating a connection to the duck db database:
conn = duckdb.connect()

#Set working directory
os.chdir('/content/drive/MyDrive/Project ADSDB Max_Julian/landing/persistent')

#Load 2 csv files and merge them
worldometer_complete = conn.execute("""SELECT * FROM read_csv_auto(['worldometer_v1_2023-10-07 12:49:49.csv',
'worldometer_v2_2023-10-07 12:49:49.csv'], union_by_name=true)""").df()



**Data Cleaning**

In [9]:
data = worldometer_complete
#Delete columns with many missing values (more than 30 %): "NewCases", 'NewDeaths', 'NewRecovered'
data = data.drop(["NewCases", 'NewDeaths', 'NewRecovered','Serious,Critical'], axis = 1)#, ])

#After some EDA, we do this:
USA_pop_2022 = 338289857 / 1000000 # source: https://www.worldometers.info/world-population/us-population/

#set None values to -1 so i can identify and change them
data['Tot Cases/1M pop'] = data['Tot Cases/1M pop'].fillna(-1)
data['Deaths/1M pop'] = data['Deaths/1M pop'].fillna(-1)
data['Tests/1M pop'] = data['Tests/1M pop'].fillna(-1)

#expert imputing (for USA). The only missings are for a part of data for USA
data['Population'] = data['Population'].fillna(USA_pop_2022)

for ind in data.index:
    if data['Tot Cases/1M pop'][ind] == -1 and data['Country,Other'][ind] == 'USA':
      data['Tot Cases/1M pop'][ind] = data['TotalCases'][ind] / USA_pop_2022
    if data['Deaths/1M pop'][ind] == -1 and data['Country,Other'][ind] == 'USA':
      data['Deaths/1M pop'][ind] = data['TotalDeaths'][ind] / USA_pop_2022
    if data['Tests/1M pop'][ind] == -1 and data['Country,Other'][ind] == 'USA':
      data['Tests/1M pop'][ind] = data['TotalTests'][ind] / USA_pop_2022

for ind in data.index: #set values back to missing
    if data['Tot Cases/1M pop'][ind] == -1:
      data['Tot Cases/1M pop'][ind] = None
    if data['Deaths/1M pop'][ind] == -1:
      data['Deaths/1M pop'][ind] = None
    if data['Tests/1M pop'][ind] == -1:
      data['Tests/1M pop'][ind] = None

#There is a negative value for 'ActiveCases' in Brunei at 2022-05-03, we replace it with the
#values from the day before in Brunei, because there are some mistakes, either in the
#'TotalCases','TotalDeaths', or 'TotalRecovered', because the value for 'ActiveCases'.
#data.loc[12373] = 'Brunei',141911.0 , 218.0, 141022.0, 671.0, 318790.0, 490.0, 717784.0, 1612436.0, 445155.0, '2022-05-03'

data.loc[12373][4] = None

<ipython-input-9-529ab4293995>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tot Cases/1M pop'][ind] = data['TotalCases'][ind] / USA_pop_2022
<ipython-input-9-529ab4293995>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Deaths/1M pop'][ind] = data['TotalDeaths'][ind] / USA_pop_2022
<ipython-input-9-529ab4293995>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tests/1M pop'][ind] = data['TotalTests'][ind] / US

**Data Imputation using interpolation**

In [12]:
pip install prophet

In [35]:
N = 200
df = data
grouped = df.groupby('Country,Other')
# Function to check if a group has more than a specified number of missing values for 'TotalDeaths'
def has_more_than_N_missing(group, N):
    return group['TotalDeaths'].isna().sum() > N

# Group by 'Country,Other'
grouped = data.groupby('Country,Other')

# Filter out countries with more than N missing values in 'TotalDeaths'
filtered_group = grouped.filter(lambda x: not has_more_than_N_missing(x, N))

# Reset the index of the resulting DataFrame
filtered_group = filtered_group.reset_index(drop=True)

# Print the filtered DataFrame
names = []
grouped = filtered_group.groupby('Country,Other')
for i in range(1,9):
  for name,g in grouped:
    if g.iloc[:,i].isna().sum() > 140:
      #names.append(g.iloc[0,0])
      #print(g.iloc[0,0], i)
      if g.iloc[0,0] not in names:
        names.append(g.iloc[0,0])
# names2 = []
# for name, group in grouped:
#     # Check if there are more than 140 missing values in any column for the group
#     if group
#     if group.drop('Country,Other', axis=1).isna().sum().max() > 140:
#         names2.append(name)
print(len(names), names)
#print(len(names2), names2)


new_data = df
names.append('Falkland Islands')
names.append('Saint Helena')
names.append('Tuvalu')
names.append('Vatican City')
names.append('DPRK')
names.append('Tajikistan')
names.append('Western Sahara')


for i in names:
  new_data = new_data.loc[new_data['Country,Other'] != i ] #not in names]
#print(new_data.isna().sum())
#Niue has too many missings in total death

16 ['Niue', 'British Virgin Islands', 'Faeroe Islands', 'French Polynesia', 'Iceland', 'Laos', 'Sierra Leone', 'St. Barth', 'Sudan', 'Tanzania', 'Tunisia', 'Comoros', 'Kiribati', 'Marshall Islands', 'Nicaragua', 'Seychelles']


In [40]:
ind = data.loc[data['Country,Other'] == 'Falkland Islands']
print(ind.iloc[:,2].isna().sum())
ind.isna().sum()

201


Country,Other         0
TotalCases            0
TotalDeaths         201
TotalRecovered      180
ActiveCases         180
Tot Cases/1M pop      0
Deaths/1M pop       201
TotalTests            0
Tests/1M pop          0
Population            0
Date                  0
dtype: int64

In [51]:
#Filter after ChatGPT solves it 19.36
# Filter the countries that have more than N missings in 1 or more columns
N = 200
df = data
grouped = df.groupby('Country,Other')
unique_country_names = set()

# Function to check if a group has more than a specified number of missing values for a given column
def has_more_than_N_missing(group, column, N):
    return group[column].isna().sum() > N
count = 0
names = []
columns = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases',
                   'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop']
for i in range(8):
  for name, group in grouped:
    if has_more_than_N_missing(group, columns[i], 100):
      names.append(name)
      #print(name)
      count+=1
print('count = ', count)
new_data = data
for i in names:
  new_data = new_data.loc[new_data['Country,Other'] != i ] #not in names]
#print(data.isna().sum(), new_data.isna().sum())

df = new_data

grouped = df.groupby('Country,Other')

# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

# Iterate through each group and impute missing values for all columns except 'Date'
for name, group in grouped:
    group.loc[:, group.columns != 'Date'] = group.loc[:, group.columns != 'Date'].interpolate()
    imputed_dataframes.append(group)

# Concatenate the imputed DataFrames back into one final DataFrame
imputed_df = pd.concat(imputed_dataframes, ignore_index=True)
print(imputed_df.isna().sum(), new_data.isna().sum())


import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman',random_state=0)

#numerical_columns = imputed_df.select_dtypes(include=['number'])

grouped_for_mice = imputed_df.groupby('Country,Other')

# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

# Iterate through each group and impute missing values for all columns except 'Date'
for name, group in grouped_for_mice:
    num_group = group.select_dtypes(include=['number'])
    num_group = imp.fit_transform(num_group)
    num_group
    imputed_dataframes.append(group)
#X=imp.fit_transform(X)
imputed_df2 = pd.concat(imputed_dataframes, ignore_index=True)
print(imputed_df.isna().sum(), imputed_df2.isna().sum())


count =  52
Country,Other         0
TotalCases            0
TotalDeaths         338
TotalRecovered      117
ActiveCases          93
Tot Cases/1M pop      0
Deaths/1M pop       338
TotalTests          300
Tests/1M pop        300
Population            0
Date                  0
dtype: int64 Country,Other          0
TotalCases             0
TotalDeaths          338
TotalRecovered      1113
ActiveCases         1089
Tot Cases/1M pop       0
Deaths/1M pop        338
TotalTests           300
Tests/1M pop         300
Population             0
Date                   0
dtype: int64


KeyError: ignored

In [ ]:
#Filter the countries that have more than N missings in 1 or more columns
N = 200
df = data
grouped = df.groupby('Country,Other')
names = []
def has_more_than_10_missing(group,number):
    return group.iloc[:,number].isna().sum() > N

for name,group in grouped:
  for i in range(1,9):
    if has_more_than_10_missing(group,i):
      if group.iat[0,0] not in names:
        names.append(group.iat[0,0])

# Function to check if a group has more than a specified number of missing values for a given column
def has_more_than_N_missing(group, column, N):
    return group[column].isna().sum() > N

# Specify the maximum number of missing values allowed for each column
max_missing_values = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases',
                   'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop']

filtered_group = {}

# Iterate through columns
for column in max_missing_values:
    filtered_group[column] = grouped.filter(lambda x: not has_more_than_N_missing(x, column, N))

combined_df = pd.concat(filtered_group.values())

# Reset the index of the combined DataFrame
combined_df = combined_df.reset_index(drop=True)

# filtered_group will now be a dictionary where each key is a column name, and the value is the filtered group for that column
print(len(names), names)
print(data.isna().sum(), combined_df.isna().sum())

In [ ]:
#try 2 for prophet
import pandas as pd
from prophet import Prophet
import numpy as np

# Create a sample DataFrame with missing values
df = new_data

grouped = df.groupby('Country,Other')

# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

# Iterate through each group and impute missing values for all columns except 'Date'
for name, group in grouped:
    group.loc[:, group.columns != 'Date'] = group.loc[:, group.columns != 'Date'].interpolate()
    imputed_dataframes.append(group)

# Concatenate the imputed DataFrames back into one final DataFrame
imputed_df = pd.concat(imputed_dataframes, ignore_index=True)

df = imputed_df

# Convert the 'Date' column to a DatetimeIndex
df['Date'] = pd.to_datetime(df['Date'])

# Group the DataFrame by the 'Country' column
grouped = df.groupby('Country,Other')

# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

columns = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases',
                   'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop']
# Iterate through each group and impute missing values using Prophet
for column_name in columns:
  for name, group in grouped:
      print('column', column_name, ' of country', group.iloc[0,0], 'we are doing now:')
      # Create a Prophet model
      model = Prophet()

      # Rename columns to fit Prophet's requirements
      group = group.rename(columns={'Date': 'ds', column_name : 'y'})

      # Remove rows with missing values as Prophet can't handle them
      group = group.dropna(subset=['y'])

      # Fit the model
      model.fit(group)

      # Create a new DataFrame with future dates to impute missing values
      future = model.make_future_dataframe(periods=len(group), freq='D')

      # Predict values for the future dates
      forecast = model.predict(future)

      # Merge the original group data with the forecasted values
      imputed_group = group.merge(forecast[['ds', 'yhat']], on='ds', how='left')

      # Rename columns back to their original names
      imputed_group = imputed_group.rename(columns={'ds': 'Date', 'yhat': column_name})

      # Fill any remaining missing values with NaN
      imputed_group[column_name] = imputed_group[column_name].fillna(np.nan)

      imputed_dataframes.append(imputed_group)


# Concatenate the imputed DataFrames back into one final DataFrame
imputed_df = pd.concat(imputed_dataframes, ignore_index=True)

# Display the imputed DataFrame
print(data.isna().sum())
print(imputed_df.isna().sum())



In [ ]:
# result:
Country,Other          0
TotalCases             0
TotalDeaths          781
TotalRecovered      3805
ActiveCases         3780
Tot Cases/1M pop       0
Deaths/1M pop        781
TotalTests          2155
Tests/1M pop        2155
Population             0
Date                   0
dtype: int64
Country,Other          0
y                      0
TotalDeaths         1264
TotalRecovered      9120
ActiveCases         9120
Tot Cases/1M pop       0
Deaths/1M pop       1264
TotalTests           352
Tests/1M pop         352
Population             0
Date                   0
TotalCases             0
dtype: int64

In [33]:

#data['Date'] = pd.to_datetime(data['Date'])

# Group the DataFrame by the 'Country' column
print(combined_df.isna().sum())

grouped = combined_df.groupby('Country,Other')


# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

# Define numeric columns for interpolation
numeric_columns = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases',
                   'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population']

# Iterate through each group and impute missing values for numeric columns
for name, group in grouped:
    for column in numeric_columns:
        # Forward fill, then backward fill missing values
        #group[column] = group[column].fillna(method='ffill').fillna(method='bfill')
        group[column] = group[column].ffill().bfill()
        # Interpolate the remaining gaps
        group[column] = group[column].interpolate()
    imputed_dataframes.append(group)


# Concatenate the imputed DataFrames back into one final DataFrame
imputed_df = pd.concat(imputed_dataframes, ignore_index=True)

# Sort the final DataFrame by 'Date'
#imputed_df = imputed_df.sort_values(by='Date')

# Display the imputed DataFrame
#print(imputed_df)
print(imputed_df.isna().sum())
print(data.isna().sum())
#Set columns to correct type
#data = imputed_df.astype({'TotalRecovered': 'int32','ActiveCases': 'int32',
               # 'Tot Cases/1M pop': 'int32','Deaths/1M pop': 'int32','TotalTests': 'int32','Tests/1M pop': 'int32', })


Country,Other           0
TotalCases              0
TotalDeaths          4238
TotalRecovered      21852
ActiveCases         21656
Tot Cases/1M pop        0
Deaths/1M pop        4238
TotalTests          11758
Tests/1M pop        11758
Population              0
Date                    0
dtype: int64
Country,Other           0
TotalCases              0
TotalDeaths          2216
TotalRecovered       6276
ActiveCases          6084
Tot Cases/1M pop        0
Deaths/1M pop        2216
TotalTests          11132
Tests/1M pop        11132
Population              0
Date                    0
dtype: int64
Country,Other          0
TotalCases             0
TotalDeaths          781
TotalRecovered      3805
ActiveCases         3780
Tot Cases/1M pop       0
Deaths/1M pop        781
TotalTests          2155
Tests/1M pop        2155
Population             0
Date                   0
dtype: int64


**Check if imputed values are reasonable**

In [ ]:
original_data = worldometer_complete
original_report = ProfileReport(original_data, title='Original Data')
#original_report.to_file("original_report.html")
transformed_report = ProfileReport(data, title="Transformed Data")
comparison_report = original_report.compare(transformed_report)
#comparison_report.to_file("original_vs_transformed.html")

In [ ]:
comparison_report

In [23]:
# Create a sample DataFrame with missing values
# data = {
#     'Country': ['A', 'B', 'A', 'B', 'C', 'A', 'C'],
#     'TotalCases': [100, None, 200, 300, 400, None, 600],
#     'TotalDeaths': [10, 20, 30, None, 40, 50, None],
#     'TotalRecovered': [80, 70, None, 90, 60, None, 100],
#     'ActiveCases': [10, None, 20, 10, 10, None, 20],
#     'Tot Cases/1M pop': [1000, None, 2000, 3000, 4000, None, 5000],
#     'Deaths/1M pop': [100, None, 200, 300, 400, None, 500],
#     'TotalTests': [5000, 6000, 7000, 8000, 9000, 10000, 11000],
#     'Tests/1M pop': [50000, 60000, 70000, 80000, 90000, 100000, 110000],
#     'Population': [1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000],
#     'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-07']
# }

#df = pd.DataFrame(data)
df = data
# Group the DataFrame by the 'Country' column
grouped = df.groupby('Country,Other')

# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

# Iterate through each group and impute missing values for all columns except 'Date'
for name, group in grouped:
    group.loc[:, group.columns != 'Date'] = group.loc[:, group.columns != 'Date'].interpolate()
    imputed_dataframes.append(group)

# Concatenate the imputed DataFrames back into one final DataFrame
imputed_df = pd.concat(imputed_dataframes, ignore_index=True)

# Sort the final DataFrame by 'Date'
#imputed_df = imputed_df.sort_values(by='Date')

# Display the imputed DataFrame
#print(imputed_df)

**Export Preprocessed data to csv-file**

In [44]:
data.to_csv('/content/drive/MyDrive/Project ADSDB Max_Julian/landing/persistent/worldometer_preprocessed_2023-10-20 16:24:36.csv')  # or True?

In [ ]:
#ActiveCases cant be negative
#ind = X.loc[X[3] < 1]
#print(ind)
#print(X[3][12373])

#data_Brunei = data.loc[data["Country,Other"] == 'Brunei']
#ind4 = ind_full.loc[ind_full['Date'] == '2022-05-04']
#ind3 = ind_full.loc[ind_full['Date'] == '2022-05-03']
#data_previous_day = data_Brunei.loc[data_Brunei['Date'] == '2022-05-02']# and data["Country,Other"] == 'Brunei']

ind = data.loc[data['Date'] == '2022-03-08']
print(ind)
#print(ind['ActiveCases'])
#for i in ind.index:
  #if ind['ActiveCases'][i] < 0:
    #print('bingo')
  #print(ind['ActiveCases'][i])
#ind['ActiveCases']
# for i in data.index:
#   if data['ActiveCases'][i] < 0:
#     print('bingo')
# data.loc[12373] = 'Brunei',141911.0 , 218.0, 141022.0, 671.0, 318790.0, 490.0, 717784.0, 1612436.0, 445155.0, '2022-05-03'
# for i in data.index:
#   if data['ActiveCases'][i] < 0:
#     print('bingo')
# print('no bingo')


In [ ]:
profile = ProfileReport(data, title="Worldometer Dataset")#, html={'style': {'full_width': True}}, sort=None)
profile

In [ ]:
## Notes

#Delete row with a lot of missing values:
#nan_rows = data.isna()
#print(len(nan_rows))
#-----------------------------
#after imputing missing values, make sure all types are correct (int instead of float):
#df = df.astype({'TotalRecovered': 'int32','ActiveCases': 'int32','Serious/Critical': 'int32',
#'Tot Cases/1M pop': 'int32','Deaths/1M pop': 'int32','TotalTests': 'int32','Tests/1M pop': 'int32', })
#possible to overwrite error, but NaN gets replaced with <NA>'s:
#df = df.astype({'Serious/Critical': 'Int64'}, errors='ignore')
#---------------------------------

df = data
indices = data.loc[databool["TotalDeaths"]==True]
totdeath = data.loc[data['ActiveCases'] + data['TotalRecovered'] - data['TotalCases'] == 0]
print(totdeath)
df["TotalDeaths"] = np.where(data['ActiveCases'] + data['TotalRecovered'] - data['TotalCases'] == 0, 0, np.nan)
#df = indices


#df = df.astype({'TotalRecovered': 'int32','ActiveCases': 'int32','Serious/Critical': 'int32',
                #'Tot Cases/1M pop': 'int32','Deaths/1M pop': 'int32','TotalTests': 'int32','Tests/1M pop': 'int32', })
#df = df.astype({'Serious/Critical': 'Int64'}, errors='ignore')
#-------------------------------------
databool = df.isna()
indices = data.loc[databool["TotalDeaths"]==True]
totdeath = data.loc[data['ActiveCases'] + data['TotalRecovered'] - data['TotalCases'] == 0]
#print(totdeath)
data["TotalDeaths"] = np.where(data['ActiveCases'] + data['TotalRecovered'] - data['TotalCases'] == 0, 0, np.nan)

Empty DataFrame
Columns: [Country,Other, TotalCases, TotalDeaths, TotalRecovered, ActiveCases, Tot Cases/1M pop, Deaths/1M pop, TotalTests, Tests/1M pop, Population, Date]
Index: []
